<a href="https://colab.research.google.com/github/AbhiRishi96/qure_ai_problems/blob/main/qure_ai_problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORT REQUIRED LIBRARIES & PROVIDING FILE PATHS

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_energy = pd.read_excel(r'/content/drive/MyDrive/Qure_ai/Question2/Energy_Indicators.xls')
df_sciem = pd.read_excel(r'/content/drive/MyDrive/Qure_ai/Question2/scimagojr_country_rank_1996-2021.xlsx')
df_gdp = pd.read_csv('/content/drive/MyDrive/Qure_ai/Question2/API_NY_GDP_MKTP_CD_DS2_en_csv_v2_4701247.csv', header=2)

### CLEANING ENERGY DATA

In [3]:
#### selecting only required columns from the complete dataset
df_energy = df_energy.iloc[17:244]

In [ ]:
#### renaming column names
df_energy = df_energy[["Unnamed: 1","Unnamed: 3", "Unnamed: 4", "Unnamed: 5" ]]
df_energy.rename(columns = {'Unnamed: 1' : 'Country', 
               'Unnamed: 3' : 'Energy Supply',
               'Unnamed: 4' : 'Energy Supply per Capita', 
               'Unnamed: 5' : '% Renewable'}, inplace = True)

In [5]:
#### replacing blanks with nan values and transforming data (cleaning data to suit our requirements)
df_energy['Energy Supply'] = df_energy['Energy Supply'].replace("...", np.nan)
df_energy['Energy Supply per Capita'] = df_energy['Energy Supply per Capita'].replace("...", np.nan)
df_energy['Energy Supply'] = df_energy['Energy Supply'].transform(lambda x: x * 1000000)
df_energy['Country'] = df_energy['Country'].replace(" ", '')


In [6]:
#### renaming country names with new names
df_energy['Country'] = df_energy['Country'].replace({'Republic of Korea' : 'South Korea',
                                                'United States of America' : "United States",
                                                'United Kingdom of Great Britain and Northern Ireland' : "United Kingdom",
                                                'China, Hong Kong Special Administrative Region' : "Hong Kong"})
 

In [7]:
#### transforming rows in country column to remove noise
df_energy['Country'] = df_energy['Country'].transform(lambda x : re.sub(r'\([^)]*\)','', x))
df_energy['Country'] = df_energy['Country'].transform(lambda x : ''.join(i for i in x if not i.isdigit()))
df_energy['Country'] = df_energy['Country'].transform(lambda x : x.rstrip())

#### CLEANING WORLD BANK DATA

In [8]:
#### renaming column name to 'country'
df_gdp.rename(columns = {'Country Name' : 'Country'}, inplace = True)
#### renaming country names with new names
df_gdp['Country'] = df_gdp['Country'].replace({'Korea, Rep.' : "South Korea",
                                               'Iran, Islamic Rep.' : "Iran", 
                                               'Hong Kong SAR, China' : "Hong Kong" })
            

#### CLEANING GDP DATA

In [9]:
#### removing unnecessary columns
df_sciem = df_sciem.drop(['Region'], axis=1)

#### MERGING THE DATASETS

In [10]:
df_energy.set_index('Country', inplace = True)  
df_gdp.set_index('Country', inplace = True)            
df_sciem.set_index('Country', inplace = True)

In [11]:
df_sciem = df_sciem[df_sciem['Rank'] <= 15] 
temp_df_sciem = df_sciem.merge(df_energy, how = 'left', on='Country')


In [12]:
def pick_cols_year(cur_year, offset, interval):
    cols_last_off = [cur_year]
    for i in range (0, offset-1):
        cur_year = str((pd.Timestamp(cur_year) - pd.DateOffset(years = interval)).year)
        cols_last_off.append(cur_year)
        
    return [e for e in reversed(cols_last_off)]
        
cols_last_ten = pick_cols_year('2021', 10, 1)

df_gdp = df_gdp[cols_last_ten]

df_final = temp_df_sciem.merge(df_gdp, how = 'left', on = 'Country')


In [13]:
df_final

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Country,,,,,,,,,,,,,,,,,,,,
China,1,303064,301778,3036531,2092737,10.02,273,1.271910e+11,93.0,19.75491,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13,1.123328e+13,1.231041e+13,1.389482e+13,1.427994e+13,1.468767e+13,1.773406e+13
United States,2,184851,181106,2623922,819242,14.19,389,9.083800e+10,286.0,11.57098,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13,1.869511e+13,1.947962e+13,2.052716e+13,2.137257e+13,2.089374e+13,2.299610e+13
India,3,60257,58589,590570,213760,9.80,198,3.319500e+10,26.0,14.96908,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12,2.294798e+12,2.651473e+12,2.702930e+12,2.831552e+12,2.667688e+12,3.173398e+12
Japan,4,52780,52281,557023,132113,10.55,203,1.898400e+10,149.0,10.23282,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12,5.003678e+12,4.930837e+12,5.037835e+12,5.123318e+12,5.040108e+12,4.937422e+12
United Kingdom,5,47141,45928,748994,132737,15.89,244,7.920000e+09,124.0,10.60047,2.719158e+12,2.803291e+12,3.087166e+12,2.956574e+12,2.722852e+12,2.699017e+12,2.900791e+12,2.878674e+12,2.756900e+12,3.186860e+12
Germany,6,42343,41464,528645,115432,12.48,211,1.326100e+10,165.0,17.90153,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12,3.469853e+12,3.690849e+12,3.977289e+12,3.888327e+12,3.846414e+12,4.223116e+12
Russian Federation,7,39424,39189,142937,67935,3.63,96,3.070900e+10,214.0,17.28868,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12,1.276787e+12,1.574199e+12,1.657330e+12,1.693114e+12,1.488322e+12,1.775800e+12
Canada,8,35588,34940,665415,113363,18.70,246,1.043100e+10,296.0,61.94543,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12,1.527995e+12,1.649266e+12,1.725329e+12,1.742015e+12,1.645423e+12,1.990762e+12
Italy,9,31260,29959,433388,105641,13.86,177,6.530000e+09,109.0,33.66723,2.086958e+12,2.141924e+12,2.162010e+12,1.836638e+12,1.877072e+12,1.961796e+12,2.091932e+12,2.011286e+12,1.892574e+12,2.099880e+12
